In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'Erythroid'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs["clusters"] = adata.obs["celltype"].to_numpy()

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20, keep_raw=True)

In [6]:
model_path_base = f'checkpoints/{dataset}_notime'
figure_path_base = f'figures/{dataset}_notime'
data_path = f'data/velovae/discrete/{dataset}_notime'
gene_plot = ['Smim1', 'Blvrb', 'Hba-x', 'Lmo2']

# Informative Time Prior

In [7]:
capture_time = adata.obs['stage'].to_numpy()
tprior = np.array([float(x[1:]) for x in capture_time])
adata.obs['tprior'] = tprior - tprior.min()

# Discrete VAE

In [8]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)

dvae = vv.model.DVAE(adata, 
                     tmax=20, 
                     dim_z=5, 
                     device='cuda:0',
                     init_method='tprior',
                     init_key='tprior',
                     tprior=None,
                     init_ton_zero=False)

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Initialization using prior time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Gaussian Prior.
Library scale (U): Max=5.92, Min=0.06, Mean=0.99
Library scale (S): Max=4.65, Min=0.26, Mean=1.05
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 54, test iteration: 106
*********       Stage 1: Early Stop Triggered at epoch 1013.       *********
*********                      Stage  2                       *********


  0%|          | 0/9815 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.006
Average Set Size: 233
*********       Stage 2: Early Stop Triggered at epoch 2138.       *********
*********              Finished. Total Time =   0 h : 21 m : 39 s             *********
Final: Train ELBO = -1193.887,           Test ELBO = -1271.903
       Training MSE = 18.295, Test MSE = 49.423


# Full VB

In [9]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)

dvae = vv.model.DVAEFullVB(adata, 
                           tmax=20, 
                           dim_z=5, 
                           device='cuda:0',
                           init_method='tprior',
                           init_key='tprior',
                           tprior=None,
                           init_ton_zero=False)

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dfullvb', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Initialization using prior time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Gaussian Prior.
Library scale (U): Max=5.92, Min=0.06, Mean=0.99
Library scale (S): Max=4.65, Min=0.26, Mean=1.05
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 54, test iteration: 106
*********       Stage 1: Early Stop Triggered at epoch 878.       *********
*********                      Stage  2                       *********


  0%|          | 0/9815 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.007
Average Set Size: 223
*********       Stage 2: Early Stop Triggered at epoch 1208.       *********
*********              Finished. Total Time =   0 h : 14 m :  9 s             *********
Final: Train ELBO = -1226.870,           Test ELBO = -1272.119
       Training MSE = 28.330, Test MSE = 55.291


# Evaluation

In [ ]:
cluster_edges = [('Blood progenitors 1', 'Blood progenitors 2'),
                 ('Blood progenitors 2', 'Erythroid1'),
                 ('Erythroid1', 'Erythroid2'),
                 ('Erythroid2', 'Erythroid3')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE', 'Discrete FullVB'],
                 ['dvae', 'dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)